# ASTR 119 Final Project #3 - Tutorial Image

Importing our packages

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.,8.]

Reading the image into a standard 2-D numpy array

In [ ]:
hdu_list = fits.open("C:\\Users\\peter\\Downloads\\image.fits")
data = hdu_list[0].data

Display the given image for the tutorial

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('Tutorial PNG files/tutorial_image')

Measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data)

Get a "global" mean and noise of the image background:

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

Evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()

Display the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('Tutorial PNG files/tutorial_image_background')

Evaluate the background noise

In [ ]:
bkg_rms = bkg.rms()

Display the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('Tutorial PNG files/tutorial_background_noise')

Subtract the background from the base image

In [ ]:
data_sub = data - bkg

Extracts the objects from the image

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Return the number of objects detected

In [ ]:
len(objects)

Circle the objects found on the background-subtracted image with a red ellipse and display the final image

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('Tutorial PNG files/tutorial_image_final')

Create arrays for the fluxes and errors of each object and print the first 10

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))